In [12]:
# -*- coding: utf8 -*-

In [13]:
import numpy as np

### 载入整理好的训练和测试数据集 

In [14]:
def rfLoadDatasets(filePath):
    x_disease_trainAll = list(open(filePath+"DIS.feature","r").readlines())
    x_admin_trainAll = list(open(filePath+"ADMIN.feature","r").readlines())
    x_discharge_trainAll = list(open(filePath+"TREAT.feature", 'r').readlines())
    y_trainAll = list(open(filePath+".label","r").readlines())
    
    return x_disease_trainAll,x_admin_trainAll, x_discharge_trainAll,y_trainAll

filePath = "../rfModel/datasets/train"
x_disease_trainAll,x_admin_trainAll,x_discharge_trainAll,labelList = rfLoadDatasets(filePath )
x_admin_trainAll = x_admin_trainAll[0:86096]
labelList = labelList[0:86096]
filePathtest = "../rfModel/datasets/test"
x_disease_testAll,x_admin_testAll, x_discharge_testAll,labelListTest = rfLoadDatasets(filePathtest)
x_admin_testAll = x_admin_testAll[0:21168]
labelListTest = labelListTest[0:21168]

In [15]:
print(len(x_disease_trainAll),len(x_admin_testAll))

92496 21168


### 计算数据集中的词典

In [16]:
def dataset2Dict(x_admin_trainAll, x_admin_testAll):
    diseaseAll = list()
    diseaseDict = dict()
    diseaseAll = x_admin_trainAll.copy()
    print("1=",len(diseaseAll))
    diseaseAll = diseaseAll+(x_admin_testAll)
    print("2=",len(diseaseAll))
    j=0
    for i in range(len(diseaseAll)):
        dis = diseaseAll[i]
        disArr = dis.split(" ")
        for ii in range(len(disArr)):
            dis1 = disArr[ii]
            if dis1  in diseaseDict:
                continue
            else:
                diseaseDict[dis1] = j 
                j +=1
    diseaseDictLen = len(diseaseDict)
    print("diseaseLen=", diseaseDictLen)
    #diseaseDictMap = dict([(diseaseDict[i].strip(), i) for i in range(len(diseaseDict))])
    #print(diseaseDictMap[],diseaseDictMap[1],diseaseDictMap[2])
    #print(diseaseDictMap['恶性肿瘤放疗'])
    
    return diseaseDict, diseaseDictLen
diseaseDictMap, diseaseDictLen = dataset2Dict(x_admin_trainAll, x_admin_testAll)
    

1= 86096
2= 107264
diseaseLen= 58798


In [17]:
#加载停用词表
import pandas as pd
def loadStopDict(fileName):
    df = pd.read_csv(fileName,sep = ' ',encoding = 'utf-8', header=None)
    #print(df)
    df.columns = ['a']
    stopDict = (df['a'].tolist())
    return stopDict
#stopDict = loadStopDict("disStop.list")
stopDict = loadStopDict('../datasets/stopCat.list')
print(len(stopDict))

34191


In [18]:
diseaseDictMapNew = dict()# diseaseDictLen
index = 0
for item in diseaseDictMap:
    if item not in stopDict:
        diseaseDictMapNew[item] = index
        index +=1
diseaseDictLen = len(diseaseDictMapNew)

KeyboardInterrupt: 

### 将关键字列表中的每个词的位序设置为1，默认为0

In [22]:
def disease2SvmDatasets(x_admin_trainAll, diseaseDictMap, diseaseDictLen ):
    disVecTrainAll = list()
    for i in range(len(x_admin_trainAll)):
        dis = x_admin_trainAll[i].strip()
        disArr = dis.split(" ")
        disVec = np.zeros((diseaseDictLen))
        if dis=='nan':
            disVecTrainAll.append(disVec)
            continue
        for ii in range(len(disArr)):
            dis1 = disArr[ii]
            #print("dis1=", dis1, dis)
            if dis1 in diseaseDictMap:
                disVec[diseaseDictMap[dis1]] = 1
        #print(i, diseaseDictMap[dis])
        disVecTrainAll.append(disVec)

    return disVecTrainAll
print(len(x_disease_trainAll))
vecList = disease2SvmDatasets(x_admin_trainAll, diseaseDictMapNew, diseaseDictLen )
#print(disVecTrainAll[0])
vecListTest = disease2SvmDatasets(x_admin_testAll, diseaseDictMapNew, diseaseDictLen )


92496


## SVM L1和L2测试

In [20]:
from svmModel import *

In [ ]:
from svmModel import *
svm_model_l1 =LinearSVCSearchL1My(vecList, labelList, vecListTest, labelListTest) 

--------------Begin SVC Train!----------------


In [21]:
svm_model_l2 =LinearSVCSearchL2My(vecList, labelList, vecListTest, labelListTest)  

-------------Begin SVC Train!------------
SVC Classifier trained in 22.112 seconds
LinearSVC(C=0.10000000000000001, class_weight=None, dual=True,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
     tol=0.0001, verbose=0)
this score: 0.76
             precision    recall  f1-score   support

        0 
       0.68      0.69      0.69      1577
        1 
       0.71      0.78      0.74      1589
       10 
       0.93      0.90      0.92      1599
       11 
       0.81      0.82      0.82      1145
       12 
       0.81      0.77      0.79      1254
       13 
       0.81      0.75      0.78      1290
        2 
       0.71      0.71      0.71      1591
        3 
       0.74      0.70      0.72      1583
        4 
       0.83      0.80      0.82      1598
        5 
       0.80      0.86      0.83      1599
        6 
       0.80      0.79      0.80      1591
        7 
       0.64      0.56

### 随机森林模型训练和效果评估

In [12]:
print(vecListTest[0])
from svmModel import *
#rfModelTrain(vecList,labelList,vecListTest,labelListTest )

[ 0.  1.  0. ...,  0.  0.  0.]
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=40, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
             precision    recall  f1-score   support

        0.0       0.51      0.46      0.48      1577
        1.0       0.56      0.66      0.61      1589
        2.0       0.50      0.57      0.54      1591
        3.0       0.58      0.59      0.59      1583
        4.0       0.45      0.75      0.56      1598
        5.0       0.76      0.83      0.79      1599
        6.0       0.68      0.67      0.68      1591
        7.0       0.45      0.31      0.36      1587
        8.0       0.59      0.55      0.57      1580
        9.0       0

In [9]:
#test dataset and return results
from time import time
def doSVCTest(formatFeaturesTestList, formatLabelTestList, svc_model):
    t0 = time()
    predictions_rf = svc_model.predict(formatFeaturesTestList)
    test_accuracy = (predictions_rf==formatLabelTestList).tolist().count(True)/float(len(formatFeaturesTestList))
    tt = time()-t0
    print ("SVC Prediction made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4)))
    return test_accuracy

In [19]:
from svmModel import *
import numpy as np

In [20]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn import svm, grid_search
punishList = punishList = [0.1,1,10]#np.arange(0.1,1.0,0.1)
bestSVCModel = None
bestTestAccuracy = 0
accuracyList = list()

for punish in punishList:
    svc_model = doSVCTrainL2(np.array(vecList,dtype= "f"),np.array( labelList,dtype="f"), punish)
    print (svc_model)
    #test_accuracy = doSVCTest(np.array(vecListTest,dtype= "f"),np.array(labelList,dtype="f"), svc_model)
    print(classification_report(np.array(labelListTest,dtype="f"),svc_model.predict(vecListTest)))
for punish in punishList:
    svc_model = doSVCTrainL1(np.array(vecList,dtype= "f"),np.array( labelList,dtype="f"), punish)
    print (svc_model)
    #test_accuracy = doSVCTest(np.array(vecListTest,dtype= "f"),np.array(labelList,dtype="f"), svc_model)
    print(classification_report(np.array(labelListTest,dtype="f"),svc_model.predict(vecListTest)))


Begin SVC Train!
SVC Classifier trained in 22.838 seconds
LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
             precision    recall  f1-score   support

        0.0       0.65      0.68      0.66      1577
        1.0       0.67      0.77      0.72      1589
        2.0       0.67      0.69      0.68      1591
        3.0       0.71      0.70      0.70      1583
        4.0       0.81      0.79      0.80      1598
        5.0       0.77      0.87      0.82      1599
        6.0       0.79      0.79      0.79      1591
        7.0       0.60      0.54      0.57      1587
        8.0       0.75      0.77      0.76      1580
        9.0       0.62      0.75      0.68      1585
       10.0       0.92      0.89      0.91      1599
       11.0       0.80      0.82      0.81      1145
       12.0       0.79      0.75     